# Hand detection

## Libraries

In [41]:
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import time

cv2.__version__

'4.10.0'

## Extracting files

In [2]:
pose_path = "pose.zip"
zip_object = zipfile.ZipFile(file = pose_path, mode = "r") 
zip_object.extractall("./")

## Defining the path to the models

In [3]:
proto_file = "pose/hand/pose_deploy.prototxt"
weights_file = "pose/hand/pose_iter_102000.caffemodel"

## Defining the number of points and the pairs of points

In [4]:
number_of_points = 22
pairs = [[0, 1], [1, 2], [2, 3], [3, 4], [0, 5], [5, 6], [6, 7], [7, 8],
              [0, 9], [9, 10], [10, 11], [11, 12], [0, 13], [13, 14], [14, 15],
              [15, 16], [0, 17], [17, 18], [18, 19], [19, 20]]

## Loading an image

In [5]:
img = cv2.imread("hand.jpg")

In [6]:
img.shape

(612, 612, 3)

In [7]:
cv2.imshow('hand', img)
cv2.waitKey(0)  # Wait for a key press to close the window
cv2.destroyAllWindows()  # Close the window

In [8]:
copy = np.copy(img)

In [9]:
width = img.shape[1]
heigth = img.shape[0]
proportion = width / heigth
proportion

1.0

In [21]:
pointA_color, pointB_color, line_color = (25, 24, 228), (25, 24, 228), (12, 192, 92)
txtpoint_color = (25, 16, 245)

font_size, line_size, circle_size, thickness_ = 5, 1, 4, 2

font = cv2.FONT_HERSHEY_SIMPLEX

## Loading the model

In [22]:
model = cv2.dnn.readNetFromCaffe(proto_file, weights_file)

## Changing the size of the image

In [23]:
new_height = 256
new_width = int((proportion*new_height)*8//8)

In [24]:
new_height, new_width

(256, 256)

## Converting the imate from cv2 to blob Caffe

In [25]:
blob_entry = cv2.dnn.blobFromImage(img, 1.0 / 255,
                                     (new_width, new_height),
                                     (0, 0, 0), swapRB=False, crop=False)

## Output

In [26]:
model.setInput(blob_entry)
output = model.forward()

In [27]:
output.shape

(1, 22, 32, 32)

## Plotting the output

In [28]:
points = []
threshold = 0.1

for i in range(number_of_points):
    confidence_map = output[0, i, :, :]
    confidence_map = cv2.resize(confidence_map, (width, heigth))

    _, confidence, _, point = cv2.minMaxLoc(confidence_map)

    if confidence > threshold:
        cv2.circle(copy, (int(point[0]), int(point[1])), 5, pointA_color,
                   thickness=thickness_, lineType=cv2.FILLED)
        cv2.putText(copy, ' ' + (str(int(point[0]))) + ',' +
                    str(int(point[1])), (int(point[0]), int(point[1])),
                    font, 0.3, txtpoint_color, 0, lineType=cv2.LINE_AA)

        cv2.circle(img, (int(point[0]), int(point[1])), circle_size,
                   pointA_color,
                   thickness=thickness_, lineType=cv2.FILLED)
        cv2.putText(img, ' ' + "{}".format(i), (int(point[0]),
                                                  int(point[1])),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.3,
                    txtpoint_color,
                    0, lineType=cv2.LINE_AA)

        points.append((int(point[0]), int(point[1])))

    else:
        points.append((0, 0))

In [29]:
len(points)

22

In [30]:
cv2.imshow('hand', img)
cv2.waitKey(0)  # Wait for a key press to close the window
cv2.destroyAllWindows()  # Close the window

## Drawning the lines

In [31]:
for pair in pairs:
    p1 = pair[0]
    p2 = pair[1]

    if points[p1] != (0, 0) and points[p2] != (0, 0):
        cv2.line(img, points[p1], points[p2], line_color,
                 line_size, lineType=cv2.LINE_AA)
        cv2.line(img, points[p1], points[p2], line_color, line_size,
                 lineType=cv2.LINE_AA)

In [32]:
cv2.imshow('img', img)
cv2.waitKey(0)  # Wait for a key press to close the window
cv2.destroyAllWindows()  # Close the window

## Detecting in a video

In [84]:
font_size, line_size, circle_size, thickness_ = 5, 3, 8, 2

In [85]:
video = "gym.mp4"
capture = cv2.VideoCapture(video)
ret, frame = capture.read()
ret

True

In [86]:
result = 'hand-detection.mp4'
save_video = cv2.VideoWriter(result, cv2.VideoWriter_fourcc(*'mp4v'), 30,
                              (frame.shape[1], frame.shape[0]))

In [87]:
frame.shape

(1280, 720, 3)

In [88]:
width = frame.shape[1]
heigth = frame.shape[0]
proportion = width / heigth
proportion

0.5625

In [89]:
new_height = 480
new_width = int((proportion*new_height)*8//8)
new_width

270

In [90]:
threshold = 0.1
while (cv2.waitKey(1) < 0):
    t = time.time()
    connected, frame = capture.read()

    if not connected:
        break
    
    frame_copy = np.copy(frame)
    size = cv2.resize(frame, (width, heigth))
    blur_map = cv2.GaussianBlur(size, (3, 3), 0, 0)
    image_background = np.uint8(blur_map > threshold)

    blob_entry = cv2.dnn.blobFromImage(frame, 1.0 / 255,
                                         (new_width, new_height),
                                    (0, 0, 0), swapRB=False, crop=False)

    model.setInput(blob_entry)
    output = model.forward()

    points = []

    for i in range(number_of_points):
        confidence_map = output[0, i, :, :]
        confidence_map = cv2.resize(confidence_map, (width, heigth))

        _, confidence, _, point = cv2.minMaxLoc(confidence_map)

        if confidence > threshold:
            cv2.circle(frame_copy, (int(point[0]), int(point[1])),
                       circle_size, pointA_color, thickness=thickness_,
                       lineType=cv2.FILLED)
            cv2.putText(frame_copy, "{}".format(i), (int(point[0]), int(point[1])),
                        font, .8,
                        txtpoint_color, 2, lineType=cv2.LINE_AA)

            points.append((int(point[0]), int(point[1])))
        else:
            points.append((0, 0))

    for pair in pairs:
        p1 = pair[0]
        p2 = pair[1]

        if points[p1] != (0, 0) and points[p2] != (0, 0):
            cv2.line(frame, points[p1], points[p2], line_color,
                     line_size, lineType=cv2.LINE_AA)
            cv2.circle(frame, points[p1], circle_size, pointA_color,
                       thickness=thickness_, lineType=cv2.FILLED)
            cv2.circle(frame, points[p2], circle_size, pointB_color,
                       thickness=thickness_, lineType=cv2.FILLED)

            cv2.line(image_background, points[p1], points[p2], line_color,
                     line_size, lineType=cv2.LINE_AA)
            cv2.circle(image_background, points[p1], circle_size, pointA_color,
                       thickness=thickness_, lineType=cv2.FILLED)
            cv2.circle(image_background, points[p2], circle_size, pointB_color,
                       thickness=thickness_, lineType=cv2.FILLED)

    key = cv2.waitKey(1)
    if key == 27:
        break

    print("Tempo total = {:.2f}seg".format(time.time() - t))
    save_video.write(frame)
save_video.release()

Tempo total = 1.21seg
Tempo total = 1.17seg
Tempo total = 1.17seg
Tempo total = 1.17seg
Tempo total = 1.16seg
Tempo total = 1.18seg
Tempo total = 1.27seg
Tempo total = 1.24seg
Tempo total = 1.22seg
Tempo total = 1.23seg
Tempo total = 1.40seg
Tempo total = 1.34seg
Tempo total = 1.22seg
Tempo total = 1.24seg
Tempo total = 1.24seg
Tempo total = 1.23seg
Tempo total = 1.23seg
Tempo total = 1.23seg
Tempo total = 1.22seg
Tempo total = 1.22seg
Tempo total = 1.23seg
Tempo total = 1.22seg
Tempo total = 1.23seg
Tempo total = 1.27seg
Tempo total = 1.30seg
Tempo total = 1.27seg
Tempo total = 1.26seg
Tempo total = 1.29seg
Tempo total = 1.23seg
Tempo total = 1.28seg
Tempo total = 1.26seg
Tempo total = 1.25seg
Tempo total = 1.30seg
Tempo total = 1.26seg
Tempo total = 1.27seg
Tempo total = 1.26seg
Tempo total = 1.27seg
Tempo total = 1.25seg
Tempo total = 1.24seg
Tempo total = 1.23seg
Tempo total = 1.27seg
Tempo total = 1.28seg
Tempo total = 1.26seg
Tempo total = 1.25seg
Tempo total = 1.25seg
Tempo tota